<a href="https://colab.research.google.com/github/petteriTeikari/minivess_mlops/blob/main/notebooks/train_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use a custom model quickly

Reuse the data(loader) from the "ML pipe"

Import the libraries needed

In [1]:
import sys
import os

If you run this Notebook on Colab, you need to install the Virtual Environment with Poetry yourself (what I understood):

In [2]:
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    print('You are running this on COLAB so installing the environment here')
    # TODO! Colab by default uses Python 3.10 so force the 3.8.17 here?
    os.chdir("/content")
    !git clone https://github.com/petteriTeikari/minivess_mlops.git
    !pip install poetry
    os.chdir("/content/minivess_mlops")
    !poetry config virtualenvs.in-project true
    # Running "!poetry install --no-ansi" is needed or not?
    !poetry install
    #!poetry shell
    # https://stackoverflow.com/a/65440080/6412152
else:
    print('Assuming that you are runnign this from IDE,\n'
          'or some other environment where you have your Jupyter kernel created from Poetry files')

Streaming output truncated to the last 5000 lines.
  • Installing itk-filtering (5.3.0): Downloading... 0%
  • Installing itsdangerous (2.1.2)
  • Installing jinja2 (3.1.2): Pending...
  • Installing joblib (1.3.2): Pending...
  • Installing google-auth (2.17.3): Installing...
  • Installing huggingface-hub (0.17.3): Installing...
  • Installing itk-filtering (5.3.0): Downloading... 0%
  • Installing itsdangerous (2.1.2)
  • Installing jinja2 (3.1.2): Pending...
  • Installing joblib (1.3.2): Pending...
  • Installing dvc (3.27.0): Installing...
  • Installing google-auth (2.17.3): Installing...
  • Installing huggingface-hub (0.17.3): Installing...
  • Installing itk-filtering (5.3.0): Downloading... 0%
  • Installing itsdangerous (2.1.2)
  • Installing jinja2 (3.1.2): Pending...
  • Installing joblib (1.3.2): Pending...
  • Installing joblib (1.3.2): Pending...
  • Installing jinja2 (3.1.2): Downloading... 0%
  • Installing joblib (1.3.2): Pending...
  • Installing joblib (1.3.2): Do

Debugger for Colab, https://zohaib.me/debugging-in-google-collab-notebook/

In [3]:
!pip install -Uqq ipdb
import ipdb
%pdb on

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00
Automatic pdb calling has been turned ON


Import the other modules now available from Poetry environment and from the Github repo (see [in_Google_Colab.ipynb](https://github.com/elise-chin/poetry-and-colab/blob/main/Using_python_poetry_in_Google_Colab.ipynb))

In [4]:
!poetry show -v # /content/minivess_mlops/.venv

Using virtualenv: /content/minivess_mlops/.venv
absl-py                  1.4.0        Abseil Python Common Libraries, see https://github.com/ab...
aiobotocore              2.7.0        Async client for aws services using botocore and aiohttp
aiohttp                  3.8.6        Async http client/server framework (asyncio)
aiohttp-retry            2.8.3        Simple retry client for aiohttp
aioitertools             0.11.0       itertools and builtins for AsyncIO and mixed iterables
aiosignal                1.3.1        aiosignal: a list of registered asynchronous callbacks
alembic                  1.12.0       A database migration tool for SQLAlchemy.
amqp                     5.1.1        Low-level AMQP client for Python (fork of amqplib).
annotated-types          0.6.0        Reusable constraint types to use with typing.Annotated
antlr4-python3-runtime   4.9.3        ANTLR 4.9.3 runtime for Python 3.7
appdirs                  1.4.4        A small Python module for determining appropr

In [5]:
if running_in_colab:
  VENV_BASE = '/content/minivess_mlops/.venv'
  VENV_PATH = os.path.join(VENV_BASE, 'lib', 'python3.10', 'site-packages')
  #VENV_PATH = "/content/gdrive/MyDrive/test-poetry/.venv/lib/python3.8/site-packages"
  #!mkdir /content/venv
  LOCAL_VENV_PATH = '/content/venv_poetry' # local notebook
  try:
    os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
  except Exception as e:
    print('symlink failed, e = {}'.format(e))
  sys.path.insert(0, LOCAL_VENV_PATH)

  sys.path.insert(0,'/content/minivess_mlops')

In [ ]:
from loguru import logger

from src.run_training import parse_args_to_dict
from src.training.experiment import define_experiment_data
from src.utils.config_utils import import_config

Import helper subfunction(s)

In [7]:
def get_dataloaders(experim_dataloaders: dict):
    # Get the "validation" and "train" dataloaders from the dictionary
    fold_name = 'fold0'
    split_names = list(experim_dataloaders[fold_name].keys())
    fold_key = experim_dataloaders.get(fold_name)
    if fold_key is not None:
        try:
            train = experim_dataloaders[fold_name]['TRAIN']
            val = experim_dataloaders[fold_name]['VAL']['MINIVESS']
        except Exception as e:
            raise IOError('Could not get the dataloaders from the dictionary, error = {}'.format(e))
    else:
        raise IOError('Fold name = "{}" not found in the dataloaders dictionary'.format(fold_name))
    return train, val

Input arguments for the training (you can add all the input arguments supported by `run_training.py` here

In [8]:
input_args = ['-c', 'tutorials/train_demo']

# Fake these as coming from the command line to match the main code (run_training.py)
sys.argv = ['notebook_run']  # Jupyter has all the extra crap, so replace that with this
for sysargv in input_args:
    sys.argv.append(sysargv)
args = parse_args_to_dict()

2023-10-26 00:17:52.011 | INFO     | src.run_training:parse_args_to_dict:73 - Parsed input arguments:
2023-10-26 00:17:52.016 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   task_config_file: tutorials/train_demo
2023-10-26 00:17:52.020 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   run_mode: train
2023-10-26 00:17:52.024 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   data_dir: /mnt/minivess-dvc-cache
2023-10-26 00:17:52.030 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   output_dir: /mnt/minivess-artifacts
2023-10-26 00:17:52.035 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   s3_mount: False
2023-10-26 00:17:52.041 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   local_rank: 0
2023-10-26 00:17:52.043 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   project_name: MINIVESS_segmentation_TEST


Create the config with Hydra from the .yaml file(s)

In [9]:
from src.utils.config_utils import config_import_script, set_up_experiment_run

config = config_import_script(args['task_config_file'],
                              parent_dir_string = '.', #f'..{os.sep}..',
                              parent_dir_string_defaults = f'..{os.sep}..')
print(config)

2023-10-26 00:17:53.772 | INFO     | src.utils.config_utils:hydra_import_config:91 - Initializing Hydra with config_path = "/content/minivess_mlops/./configs", job_name = "config_test", version_base = "1.2"
2023-10-26 00:17:53.782 | INFO     | src.utils.config_utils:hydra_import_config:93 - Hydra overrides list = ['+tutorials=train_demo']


{'VERSION': '0.01alpha', 'NAME': 'train_demo', 'config': {'VERBOSE': True, 'DATA': {'DATA_SOURCE': {'DATASET_NAMES': ['MINIVESS'], 'MINIVESS': {'FETCH_METHOD': 'DVC', 'FETCH_METHODS': {'DVC': {'datapath_base': 'data', 'version': 'placeholder'}, 'EBrains': {'DATA_INFO_URL': 'https://doi.org/10.25493/HPBE-YHK', 'DATA_DOWNLOAD_URL': 'https://data.kg.ebrains.eu/zip?container=https://data-proxy.ebrains.eu/api/v1/buckets/d-bf268b89-1420-476b-b428-b85a913eb523'}}, 'SPLITS': {'NAME': 'RANDOM', 'RANDOM': {'SEED': 42, 'TEST_VAL_RATIO': 0.1, 'TRAIN_RATIO': 0.8}, 'TXT_FILE': {'PLACEHOLDER': 0}}, 'SUBSET': {'NAME': 'ALL_SAMPLES', 'ALL_SAMPLES': {'PLACEHOLDER_PARAM': 'placeholder'}, 'DEBUG': {'TRAIN': 2, 'VAL': 2, 'TEST': 1}}, 'DATASET': {'NAME': 'MONAI_CACHEDATASET', 'MONAI_CACHEDATASET': {'CACHE_RATE': 0.0, 'NUM_WORKERS': 1}}, 'TRANSFORMS': {'TRAIN': 'BASIC_AUG', 'VAL': 'NO_AUG', 'TEST': 'NO_AUG'}}}, 'DATALOADER': {'SKIP_DATALOADER': False, 'NAME': 'DATALOADER_BASIC', 'DATALOADER_BASIC': {'PLACEHO

In [10]:
if running_in_colab:
  if os.path.exists('/mnt'):
    os.makedirs(args['data_dir'], exist_ok=True)  # DVC Cache
    os.makedirs(args['output_dir'], exist_ok=True)  # Artifacts output
  

Authenticate to AWS S3?
* [Dvc and gcp/colab tricks - Data Version Control](https://discuss.dvc.org/t/dvc-and-gcp-colab-tricks/584)
* [manage_data.ipynb - Colaboratory](https://colab.research.google.com/github/khuyentran1401/Efficient_Python_tricks_and_tools_for_data_scientists/blob/master/Chapter5/manage_data.ipynb)
* [DagsHub integration with Google Colab](https://dagshub.com/docs/integration_guide/google_colab/)

In [ ]:
''*

Pull the data from DVC

In [ ]:
!dvc pull

In [11]:
cfg = import_config(args=args, task_cfg_name=args['task_config_file'])

2023-10-26 00:17:53.825 | WARNING  | src.utils.config_utils:set_up_environment:323 - No Nvidia CUDA GPU found, training on CPU instead!
2023-10-26 00:17:53.833 | WARNING  | src.utils.config_utils:set_up_experiment_run:157 - Skipping realtime AWS S3 write, and writing run artifacts to a local non-mounted dir
2023-10-26 00:17:53.840 | DEBUG    | src.utils.config_utils:get_mounts_from_args:449 - Getting mount names from the args
2023-10-26 00:17:53.847 | INFO     | tests.env.mount_tests:debug_mounts:9 - Username = None, UID = 0, GID = 0
2023-10-26 00:17:53.851 | DEBUG    | tests.env.mount_tests:debug_mounts:12 - MOUNT: volumes/minivess-dvc-cache
2023-10-26 00:17:53.857 | DEBUG    | tests.env.mount_tests:debug_mounts:19 -  owned by root:root (owner:group)
2023-10-26 00:17:53.860 | DEBUG    | tests.env.mount_tests:debug_mounts:22 -  owned by 0:0 (owner:group)
2023-10-26 00:17:53.864 | DEBUG    | tests.env.mount_tests:debug_mounts:23 -  mount permissions: 0755
2023-10-26 00:17:53.870 | DEBUG

Import the dataloaders (now the data augmentations are here as well as data transformations)

In [12]:
_, _, experim_dataloaders, cfg['run'] = define_experiment_data(cfg=cfg)

# Get the "validation" and "train" dataloaders from the dictionary
train, val = get_dataloaders(experim_dataloaders)

OSError: ignored

Now you are ready to train your new model that you just wanna quickly test without
wanting to have a battle with the config .YAML files

Add maybe some fastai demo with MLflow autologging:
[https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py](https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py)

In [ ]:
# Iterate the dataloaders for demo
no_of_epochs = 3
logger.info('Training for {} epochs'.format(no_of_epochs))
for epoch in range(no_of_epochs):

    logger.info('Epoch {}/{}'.format(epoch, no_of_epochs - 1))

    # Train
    logger.info('train with {} batches'.format(len(train)))
    for i, batch in enumerate(train):
        images, mask = batch['image'], batch['label']

    # Validation
    logger.info('validate with {} batches'.format(len(train)))
    for j, batch in enumerate(val):
        images, mask = batch['image'], batch['label']

logger.info('Training done!')